In [1]:
#data input 
import pandas as pd
import numpy as np
from pandas import DataFrame
from sklearn.model_selection import StratifiedKFold
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop
from sklearn.model_selection import StratifiedKFold

path = 'trainrat_new.txt'


data = pd.read_csv(path,sep=' ')
print(data.head())

x = DataFrame(data.drop(["label"], axis=1))

print(x.head())

y = pd.DataFrame()

for row in data["label"]:
    tmp_se = pd.Series( [ row == "W", row == "D" , row == "L"],index = ["W", "D", "L"])
    y = y.append( tmp_se, ignore_index=True )

print( y.head() )

Using TensorFlow backend.


      label     HHATT     HHDEF     HAATT     HADEF     AHATT     AHDEF  \
13680     D -0.013286  0.055444 -0.032403  0.112291 -0.013286  0.163329   
2729      L  1.006109  0.230130  0.488624  0.489483  0.985480  0.083888   
26413     D -0.034506  0.105019  0.012418  0.435008 -0.061290  0.057348   
8975      D  4.744033 -1.088280  1.720375 -1.439166  0.698823  0.749711   
13324     W  0.016564 -0.053097  0.040713  0.194267  0.026022  0.175712   

          AAATT     AADEF  
13680 -0.032403 -0.015245  
2729   0.432996 -0.060702  
26413  0.395053  0.766563  
8975   0.396043  1.203952  
13324  0.040616  0.915641  
          HHATT     HHDEF     HAATT     HADEF     AHATT     AHDEF     AAATT  \
13680 -0.013286  0.055444 -0.032403  0.112291 -0.013286  0.163329 -0.032403   
2729   1.006109  0.230130  0.488624  0.489483  0.985480  0.083888  0.432996   
26413 -0.034506  0.105019  0.012418  0.435008 -0.061290  0.057348  0.395053   
8975   4.744033 -1.088280  1.720375 -1.439166  0.698823  0.749711

In [3]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

# define X-fold cross validation
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)


In [4]:
# create model
modelArelu = Sequential()
modelArelu.add(Dense(10, activation='relu'))
modelArelu.add(Dense(10, activation='relu'))
modelArelu.add(Dense(3, activation='softmax'))

modelArelu.compile(loss='binary_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.


In [5]:
cvscores = []
batch_size = 16
num_classes = 10
epochs = 1
for train, test in kfold.split(x, y):
    # Fit the model
    modelArelu.fit(X[train], keras.utils.to_categorical(Y[train], num_classes),
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1)


    # Evaluate
    scores = model.evaluate(X[test], keras.utils.to_categorical(Y[test], num_classes), verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)

print("%.2f%% (+/- %.2f%%)" % (numpy.mean(cvscores), numpy.std(cvscores)))


ValueError: Supported target types are: ('binary', 'multiclass'). Got 'multilabel-indicator' instead.

In [1]:
'''Trains a simple deep NN on the MNIST dataset.
Gets to 98.40% test accuracy after 20 epochs
(there is *a lot* of margin for parameter tuning).
2 seconds per epoch on a K520 GPU.
'''

from __future__ import print_function

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop
from sklearn.model_selection import StratifiedKFold
import numpy

fold_num = 10
batch_size = 128
num_classes = 10
epochs = 20

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(784,)))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))

model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Using TensorFlow backend.


11493376/11490434 [==============================] - 3s 0us/step
60000 train samples
10000 test samples
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               401920    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)          